### Import necessary Libraries


In [36]:
# Importing Libraries
import pandas as pd
import numpy as np
import re
import string

### Load the dataset ("Customer_reviews.xlsx")


In [37]:
df1 = pd.read_excel("Customer_reviews.xlsx")
print(df1.shape)
df1.head()

(100000, 7)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:00
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:00
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:00
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:00
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:00


In [38]:
# In sentiment analysis we don't need review_id, order_id, review_creation_date, review_answer_timestamp..so we have to drop those columns. for further queries, we keep order_id columns in our dataframe

df1.drop(["review_creation_date", "review_answer_timestamp", "review_id"],axis=1,inplace=True)

In [39]:
df1.head()

,order_id,review_score,review_comment_title,review_comment_message
0,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN
1,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN
2,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN
3,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...


### There are some rows those may have only review_comment_title values instead of review_comment_message values. So we have to take review_comment_title values as Reviews in those columns.
### So we have to concat review_comment_title and review_comment_message column in one column 
### To do this, we have to handle NaN values first



In [40]:
# we can not join null values and string, So converting null values into " " for solving concatenating problem of  "review_comment_title" and "review_comment_message" columns

df1.fillna('', inplace = True)

In [41]:
# converting review_comment_title and review_comment_message data into string

df1['review_comment_message'] = df1['review_comment_message'].apply(str)
df1['review_comment_title'] = df1['review_comment_title'].apply(str)

In [42]:
# Joining review_comment_title and review_comment_message columns and create a new reviews column

df1['reviews'] = df1['review_comment_title'] + ' ' + df1['review_comment_message']

In [43]:
df1.sample(5)

,order_id,review_score,review_comment_title,review_comment_message,reviews
48653,e104196e46cab854b8dc6f0beaa84d13,5,,,
77040,f825879441f1f289d12c308982d24315,5,,,
12245,464bdc55a2b4838f5d83f20f181ff858,5,,Produto chegou corretamente.,Produto chegou corretamente.
80418,926e667aed4b5a389872a0777bceb8c5,5,,,
64492,e0f06a57d717562c49f91cd309be4598,5,,,


In [44]:
# Now we can remove review_comment_title and review_comment_message columns

df1.drop(["review_comment_title","review_comment_message"], axis=1, inplace=True)

### Since we are analysing customer satisfaction from their Revies so we can drop those Rows where reviews are not present

In [45]:
# Replacing " " into null value in reviews column
 
df2 = df1.replace(" ", np.NaN)
df2.head()

,order_id,review_score,reviews
0,73fc7af87114b39712e6da79b0a377eb,4,NaN
1,a548910a1c6147796b98fdf73dbeba33,5,NaN
2,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN
3,658677c97b385a9be170737859d3511b,5,Recebi bem antes do prazo estipulado.
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,Parabéns lojas lannister adorei comprar pela ...


In [46]:
# Removing rows where NaN values are present

df2.dropna(inplace=True)

In [47]:
df2.head()

,order_id,review_score,reviews
3,658677c97b385a9be170737859d3511b,5,Recebi bem antes do prazo estipulado.
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,Parabéns lojas lannister adorei comprar pela ...
9,b9bf720beb4ab3728760088589c62129,4,recomendo aparelho eficiente. no site a marca ...
12,9d6f15f95d01e79bd1349cc208361f09,4,"Mas um pouco ,travando...pelo valor ta Boa.\n"
15,e51478e7e277a83743b6f9991dbfa3fb,5,"Super recomendo Vendedor confiável, produto ok..."


### Removing Duplicate values

In [48]:
# checking duplicated Data according to columns

count1= sum(df2.duplicated("order_id"))
count2 = sum(df2.duplicated("reviews"))
print("duplicate_order_id_count {} \nduplicate_review_count {}".format(count1,count2))

duplicate_order_id_count 153 
duplicate_review_count 4809


In [49]:
# remove duplicate value from each column
df3 = df2.copy()
df4 = df3[~(df1.duplicated("order_id"))] # removing duplicate value from  order_id column
df5 = df4[~(df1.duplicated("reviews"))] # removing duplicate value from  review column

ipykernel_launcher:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
ipykernel_launcher:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [50]:
print(df5.shape)
print("Duplicated value count in dataset {}".format(sum(df5.duplicated())))

(38573, 3)
Duplicated value count in dataset 0


### Apply First level Reviews text cleaning

In [51]:
# Removing emotion icons, symbols & pictographs, transport & map symbols, flags (iOS) etc.

def text_cleaner1(text):
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emotion icons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
                              "]+", flags=re.UNICODE)
    
    return(emoji_pattern.sub(r'', text))

In [52]:
df6 = df5.copy()

In [53]:
df6['reviews'] = df6['reviews'].apply(lambda x:text_cleaner1(x))

### Apply Secound level Reviews text cleaning

In [54]:
#This function converts to lower-case, removes square bracket, removes numbers and punctuations

def text_cleaner2(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

cleaner = lambda x: text_cleaner2(x)

In [55]:
df6['reviews'] = df6['reviews'].apply(cleaner)

In [56]:
df6.sample(7)

,order_id,review_score,reviews
14972,87f83d5d5bb0b9a13ec25c61c4a71361,3,obrigado
48725,6438d704a058758bff33c039c32f655f,3,até hoje não recebi o produto por isso não po...
57156,9c43b12ecec2272d35dcbec5a8b833e1,4,processo de entrega muito demorado
32607,16a182d2e328e5f0b3252d367e17bc14,5,satisfeito exatamente como no anúncio \nentreg...
82354,21e751f0d78afdfe36cb0e8fe45360d8,5,claro é super confiável esta de parabéns
14881,5c2c3051e99aa85bb1f2260f51e59f49,2,pedido com toalhas não recebi ainda esse pedido
93081,32fc3f3218889905a42a419486582f90,4,ótimo produto funciona muito bem não tive nenh...


## save df6 as cleaned_reviews_data dataset

In [57]:
df6.to_excel("cleaned_reviews_data.xlsx", index=False)